In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import yfinance as yf
import scipy.optimize as sc

from tracker.main import get_portfolio

ModuleNotFoundError: No module named 'tracker'

In [ ]:
RFR = 0.00

In [ ]:
def get_data(coins, start, end):
    coin_data = yf.download(coins, start=start, end=end)
    coin_data = coin_data['Close']
    returns = coin_data.pct_change()
    mean_returns = returns.mean()
    cov_matrix = returns.cov()
    return mean_returns, cov_matrix

def port_performance(weights, mean_returns, cov_matrix):
    expected_returns = np.sum(mean_returns * weights) * 365
    expected_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(365)
    return expected_returns, expected_volatility

def negative_sharpe(weights, mean_returns, cov_matrix, rfr=RFR):
    expected_returns, expected_volatility = port_performance(weights, mean_returns, cov_matrix)
    sharpe_ratio = (expected_returns - rfr) / expected_volatility
    neg_sharpe = -1 * sharpe_ratio
    return neg_sharpe

def maximize_sharpe(mean_returns, cov_matrix, bound=(0,1)):
    num_coins = len(mean_returns)
    initial_weights = num_coins * [1.0 / num_coins]
    arguments = (mean_returns, cov_matrix)
    bounds = tuple(bound for coin in range(num_coins))
    constraints = ({"type": "eq", "fun": lambda weights: np.sum(weights) - 1})
    result = sc.minimize(negative_sharpe, initial_weights, method='SLSQP', args=arguments, bounds=bounds, constraints=constraints)
    return result

In [ ]:
coins = []

for i in range(len(coins)):
    coins[i] = coins[i] + '-USD'

In [ ]:


end_date = dt.datetime(2021, 1, 23)
start_date = dt.datetime(2021, 1, 1)

# end_date = dt.datetime.now()
# start_date = end_date - dt.timedelta(days=365*3)

mean_returns, cov_matrix = get_data(coins, start_date, end_date)

In [ ]:
result = maximize_sharpe(mean_returns, cov_matrix)
print(result)
vector = result['x']
optimal_weights = [{ticker:round(x, 3)} for ticker,x in zip(tickers,vector)] 
print(optimal_weights)

In [ ]:
# x: array([0.35360146, 0.34451539, 0.23856027, 0.06332288])
# [{'CEVA': 0.354}, {'GOOGL': 0.345}, {'TSLA': 0.239}, {'ZOM': 0.063}]